In [1]:
from dask.distributed import Client, progress
from dask import delayed

client = Client(threads_per_worker=4, n_workers=1)

In [2]:
client

Client Scheduler: tcp://127.0.0.1:62619 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 4 Memory: 8.59 GB


In [3]:
from time import sleep

@delayed
def inc(x):
    sleep(1)
    return x + 1

@delayed
def add(x, y):
    sleep(1)
    return x + y

In [16]:
%%time

x = inc(1)
y = inc(2)
z = add(x, y)

CPU times: user 118 ms, sys: 29 ms, total: 147 ms
Wall time: 3.01 s


In [5]:
%%time
# This runs immediately, all it does is build a graph

x = delayed(inc)(1)
y = delayed(inc)(2)
z = delayed(add)(x, y)

CPU times: user 1.06 ms, sys: 926 µs, total: 1.98 ms
Wall time: 1.66 ms


In [15]:
%%time

z.compute()

CPU times: user 94.2 ms, sys: 23.8 ms, total: 118 ms
Wall time: 2.04 s


5

In [198]:
class Constraint:
    def __init__(self, fnc, fixed_indicies):
        self.fnc = fnc
        self.fixed_indicies = fixed_indicies

    def get_func(self):
        return self.fnc
    
    def execute(self, X, indicies):
        params = tuple([X]) + tuple([indicies[fi] for fi in self.fixed_indicies])
        return delayed(self.fnc)(*params)


class Model:
    def __init__(self, constraints):
        self.constraints = constraints
        
    def execute(self, X, indicies):
        return delayed(all)([c.execute(X, indicies) for c in self.constraints])

In [199]:
eligibility = lambda X, c, u: c+1

In [200]:
z_e = delayed(eligibility)('',1,2)

In [201]:
z_e.compute()

2

In [202]:
c1 = Constraint(eligibility, (0,1,))

In [203]:
zc1 = c1.execute('',(0,1,))
zc1.compute()

1

In [204]:
mdl = Model([
    Constraint(eligibility, (0,1,)),
    Constraint(eligibility, (0,1,)),
    Constraint(eligibility, (0,1,)),
    Constraint(eligibility, (0,1,)),
    Constraint(eligibility, (0,1,)),
])

In [205]:
z=delayed(mdl.execute)('',(0,1,))

In [206]:
z

Delayed('execute-008ec887-79c8-48f4-8581-80b492413de0')

In [207]:
%%time
z.compute().compute()

CPU times: user 26.1 ms, sys: 3.88 ms, total: 30 ms
Wall time: 51.9 ms


True

In [5]:
from greedy import Constraint

ModuleNotFoundError: No module named 'greedy'